In [ ]:
import torch
import sys
import os
from pathlib import Path

# append the current path
ROOT = Path(__file__).resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.app(str(ROOT))

from utils import *

